# Estimation Techniques for Panel Data

## 0. Introduction

Our aim in this notebook is to study different options of how a panel data model can be specified and estimated in order to use one of them in our work about the network approach in macroeconomics. We will start with the usual form, which is the most likely to be eventually used in the work: **GMM**, specificly *System GMM*, which behaves better with few industries. 
 
 Then, we will explore two more options. One coming from the engineering: **State-space Models** and one form data science: **Neural Network Models**.


## 1. System GMM

References for this chapter are, basically, a Marcelo Soto's paper: ***System GMM estimation with one small sample***.

### 1.1 Theory

The econometric model with the parameters we would like to estimate is specified as follows:
 
 $y_{it}=\alpha y_{it-1}+\beta x_{it}+\eta_i+u_{it}$ (1)
 
 i.e., we have an AR model with one aditional regressor. The variable $x_it$ is also assumed to follow an AR process. _(Here lies the endogeneity)_
 
 $x_{it}=\rho x_{it-1} + \tau\eta_i + \theta u_{it} + e_{it}$ (2)
 
 both with $i=1,\dots,N,\;\;t=1,\dots,T\;\;and\;\;|\rho|<1$. 
  
 Some usual properties are needed: 
  
  $E(\eta_i)=E(u_{it})=E(\eta_iu_{it})=E(u_{is}u_{it})=E(e_{it})=E(\eta_ie_{it})=0$  
   
  (...)
   
  *Complete with the **Marcelo Soto's** [original paper](https://www.barcelonagse.eu/sites/default/files/working_paper_pdfs/395.pdf) if needed*.
 

### 1.2 Python Implementation

In [16]:
import pandas as pd
import numpy as np

In [17]:
#### INPUTS: 
###########: y = output (dep var) matrix (NxT)
###########: x = input (ind var) matrix (NxT)

In [502]:
### data samples: (N=3, T=6)
x=[[1,2,3,3,2,4],[2,3,4,2,1,3],[1,4,5,6,2,4]]
y=[[2,4,3,6,9,4],[1,2,3,4,5,7],[8,5,1,2,4,7]]

In [19]:
def diff(x): # works
    ac=[]
    i=1
    while i<len(x):
        ac.append(x[i]-x[i-1])
        i=i+1
    return ac

In [273]:
def z_li(i,x,y):        # WORKS FINE
    z=[]
    vxi=diff(x[i])
    vyi=diff(y[i])
    pos=0
    while pos<(len(vxi)-1):
        ac=[0]*(len(vxi)-1)*2
        ac[pos*2]=vyi[pos]
        ac[pos*2+1]=vxi[pos]
        z.append(ac)
        pos=pos+1
    return z


In [21]:
def tr(M):           # transponse
    trans=[]
    for j in range(len(M[0])):
        ac=[]
        for i in range(len(M)):
            ac.append(M[i][j])
        trans.append(ac)
    return trans

In [281]:
def z_l(x,y):  # WORKS FINE (cont) 
    z_l0=tr(z_li(0,x,y))
    i=1
    while i<len(x):
        zli=tr(z_li(i,x,y))
        t=0
        while t<len(z_l0):
            z_l0[t]=z_l0[t]+zli[t]
            t=t+1
        i=i+1
    return tr(z_l0)

In [260]:
def z_di(i,x,y):    # WORKS FINE
    yi=y[i]
    xi=x[i]
    T=len(xi)
    z=[]
    for i in range(T-2):
        z.append([0]*(T-1)*(T-2))
    z[0][0]=yi[0]
    z[0][1]=xi[0]
    pos=1
    num=2
    emp=2
    while pos<len(z):
        i=emp
        while i<((emp+(num+2)/2)-1):
            z[pos][i]=yi[i-emp]
            i=i+1
        z[pos][int((emp+(num+2)/2))-1]=yi[pos]
        i=int((emp+(num+2)/2))
        while i<(emp+num+1):
            z[pos][i]=xi[i-int((emp+(num+2)/2))]
            i=i+1
        z[pos][emp+num+1]=xi[pos]
        num=num+2
        emp=emp+num
        pos=pos+1
        
    return z


In [254]:
def z_d(x,y): # works fine
    z_l0=tr(z_di(0,x,y))
    i=1
    while i<len(x):
        zli=tr(z_di(i,x,y))
        t=0
        while t<(len(z_l0)):
            z_l0[t]=z_l0[t]+zli[t]
            t=t+1
        i=i+1
    return tr(z_l0)

In [236]:
def w_l(x,y): # here we start to work with arrays
    zli=np.array(z_li(0,x,y))
    tzli=np.array(tr(z_li(0,x,y)))
    w1=tzli@zli
    for i in range(len(x)-1):
        zli=np.array(z_li(i+1,x,y))
        tzli=np.array(tr(z_li(i+1,x,y)))
        sumparcial=tzli@zli
        w1=w1+sumparcial
    return w1
    

In [237]:
def G_matrix(l): # l = len
    G=[]
    i=0
    while i<l:
        j=0
        ac=[]
        while j<l:
            try:
                if i==j:
                    ac.append(2)
                elif i==(j-1):
                    ac.append(-1)
                elif i==(j+1):
                    ac.append(-1)
                else:
                    ac.append(0)
                j=j+1
            except:
                ac.append(0)
                j=j+1
        G.append(ac)
        i=i+1
    return G

In [238]:
def w_d(x,y):
    zdi=np.array(z_di(0,x,y))
    T=len(x[0])
    G=np.array(G_matrix(T-2))
    tzdi=np.array(tr(z_di(0,x,y)))
    wd=tzdi@G@zdi
    for i in range(len(x)-1):
        zdi=np.array(z_di(i+1,x,y))
        tzdi=np.array(tr(z_di(i+1,x,y)))
        sumparcial=tzdi@G@zdi
        wd=wd+sumparcial
    return wd
        

In [545]:
def w_s(x,y): ## puede que esté mal !! (asegurarse!!!)
    wd=w_d(x,y)
    wl=w_l(x,y)
    ws=pd.DataFrame()
    for i in range(len(wd)):
        a=[]
        a.append(wd[i])
        b=np.array([0]*len(wl))
        ac=np.append(a,b)
        ws[i]=ac
    for i in range(len(wl)):
        a=np.array([0]*len(wd))
        b=wl[i]
        ac=np.append(a,b)
        ws[i+100]=ac
    return np.array(ws) 
            
        

In [425]:
def z_s(x,y):
    zd=np.array(z_d(x,y))
    zl=np.array(z_l(x,y))
    a=np.array([[0]*len(tr(zl))]*len(zd))
    firstpart=np.concatenate((zd,a),axis=1)
    b=np.array([[0]*len(tr(zd))]*len(zl))
    secondpart=np.concatenate((b,zl),axis=1)
    zs=np.concatenate((firstpart,secondpart),axis=0)
    return zs
    

In [461]:
def stock_x(x): #rellenar (tiene que ser 24x2) np.array
    
    return x

In [462]:
def stock_y(y): #rellenar (tiene que ser 24x1) np.array
    
    return y

In [512]:
def vec_est(x,y):
    xs=stock_x(x)
    ys=stock_y(y)
    inter0=np.array(tr(xs))@z_s(x,y)
    inter1=inter0@np.linalg.inv(w_s(x,y))
    inter2=inter1@np.array(tr(z_s(x,y)))@xs
    primera=np.linalg.inv(inter2)
    segunda=np.array(tr(xs))@z_s(x,y)@np.linalg.inv(w_s(x,y))@np.array(tr(z_s(x,y)))@ys
    bs=primera@segunda
    i=0
    while i<len(bs):
        if i==0:
            print("ESTIMADOR PARA ALPHA: " +str(bs[i]))
        else:
            print("ESTIMADOR PARA BETA "+str(i)+": "+str(bs[i]))
        i=i+1
    return bs

_(Cuando funcione correctamente todo esto se eliminará de aquí)_

### 1.3 Use of pygmm

In [548]:
import pygmm as gmm

In [550]:
gmm.vec_est(x,y) # da error porque está sin acabar

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 24 is different from 3)

#### neural networks en otro notebook (con env=panel_data_nn)